# Heatmap

In [ ]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import KFold
import datetime as dt
import warnings
from random import sample 
warnings.filterwarnings('ignore')
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

import random
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import folium
from folium.plugins import HeatMap
from folium import plugins


In [ ]:
df_covid = pd.read_csv("covid_de.csv")
df_demo = pd.read_csv("demographics_de.csv")
df_des = pd.read_excel("location.xlsx")

#drop NaN and change to datetime
df_covid = df_covid.dropna()
df_covid['date'] = pd.to_datetime(df_covid.date)


#Changing gender in demografic to have same values as covid data set. 
df_demo['gender'] = df_demo['gender'].map({'female': 'F','male':'M'})


#Merging the two dataset on state, age_group and gender. 
merged = pd.merge(df_covid,df_demo,how='inner', on=["state","age_group","gender"])
#And merging latitude and longtitude on the full dataset
df = pd.merge(merged,df_des,how='inner',on=['county'])

In [ ]:
#We start by plotting the map of Germany. 
map_DE=folium.Map([51, 9.8], zoom_start=6,tiles='Stamen Toner')
folium.Marker([51, 9.8], popup='Germany').add_to(map_DE)
map_DE

In [ ]:
#Now we plot the data as a cirkelmarker, 
for lat,lon in zip(df['Lat'],df['Lon']):
     folium.CircleMarker([lat,lon],
                            fill_color='red',
                            fill=True,
                            radius = 5,
                            max_zoom = 10,
                            fill_opacity=0.7,
                           ).add_to(map_DE)
map_DE

In [ ]:
# Adding latitude and longitude to states
state_loc = pd.DataFrame(np.array([
['Baden-Wurttemberg',48.99999229,8.999993448],
['Bayern',49.26660748,10.96660681],
['Berlin',52.52181866,13.40154862],
['Brandenburg',51.7704175,13.72996741],
['Bremen',53.05043805,8.779982461],
['Hamburg',53.55002464,9.999999144],
['Hessen',50.30000694,9.200029662],
['Mecklenburg Vorpommern',53.63330408,12.41669861],
['Niedersachsen',52.1299986,9.720004434],
['Nordrhein Westfalen',51.52998822,8.030011351],
['Rheinland Pfalz',49.58247246,7.703219156],
['Saarland',49.25039044,6.970003213],
['Sachsen',50.82998395,12.91997595],
['Sachsen Anhalt',52.13042137,11.62000362],
['Schleswig Holstein',54.28374778,9.433315388],
['Thuringen',50.97005292,11.02996212]]),columns=['State','Lat','Long'])
state_loc = state_loc.sort_values('State')

In [ ]:
state_loc['Total Cases'] = df.groupby('state')['cases'].sum().values
state_loc['Total Deaths'] = df.groupby('state')['deaths'].sum().values
state_loc['Total Recovered'] = df.groupby('state')['recovered'].sum().values
state_loc
map_DE=folium.Map([51, 9.8], zoom_start=6,tiles='Stamen Toner')
for i in range(state_loc.shape[0]):
    icon = folium.Icon(color="red")
    folium.Marker([state_loc['Lat'][i],state_loc['Long'][i]],
                  tooltip='State: {} <br>' 'Total Cases: {} <br>' 'Total Deaths: {}<br>' 'Total Recovered: {}'.format(state_loc['State'][i],state_loc['Total Cases'][i],state_loc['Total Deaths'][i],state_loc['Total Recovered'][i]),icon=icon).add_to(map_DE)
map_DE

In [ ]:
# Add total cases to df_des
df_des['Total Cases']= df.groupby('county')['cases'].sum().values

# Create new dataframe 
df_county = df_des[['county','Lat','Lon']]

# Add rows equal to total cases pr. county
for i in range(df_des.shape[0]):
    df_county=df_county.append([df_county[i:i+1]]*(df_des['Total Cases'][i]-1))

In [ ]:
# The two plots together 


#A new map is defined: 
map_DE_heat=folium.Map([51.2, 9.8], zoom_start=6,tiles='Stamen Toner')

# Ensureing that the data is float
df_county['Lon'] = df_county['Lon'].astype(float)
df_county['Lat'] = df_county['Lat'].astype(float)

# Filter the DataFrame for columns, so only latitude and longtitude is remaining 
heat_df = df_county[['Lat', 'Lon']]

# List comprehension to make out list of lists
heat_data = [[row['Lat'],row['Lon']] for index, row in heat_df.iterrows()]

# Plot it on the map. 
#The parameter radius is set to 20 meters and the max_zoom level to 20 for the tile layer that is created.
HeatMap(data=heat_data,radius=20, max_zoom=20).add_to(map_DE_heat)


state_loc['Total Cases'] = df.groupby('state')['cases'].sum().values
state_loc['Total Deaths'] = df.groupby('state')['deaths'].sum().values
state_loc['Total Recovered'] = df.groupby('state')['recovered'].sum().values

for i in range(state_loc.shape[0]):
    icon = folium.Icon(color="red")
    folium.Marker([state_loc['Lat'][i],state_loc['Long'][i]],
                  tooltip='State: {} <br>' 'Total Cases: {} <br>' 'Total Deaths: {}<br>' 'Total Recovered: {}'.format(state_loc['State'][i],state_loc['Total Cases'][i],state_loc['Total Deaths'][i],state_loc['Total Recovered'][i]),icon=icon).add_to(map_DE_heat)




# Display the map
map_DE_heat




In [ ]:
df_copy = df

In [ ]:
df1 = pd.DataFrame(columns=['state','county','age_group','gender','date','cases','deaths','population','Lat','Lon'])

In [ ]:

#df_copy = df[(df[['cases']] != 0).all(axis=1)]
df_copy = df[~(df[['cases']] == 0).any(axis=1)]
#df_copy.drop(df_copy[df_copy['cases'] < 1].index, inplace = True) 
#df_copy['cases'] = df_copy['cases'].astype(int)

for i in range(df_copy.shape[0]):
    print(i)
    df1=df1.append([df_copy[i:i+1]]*df_copy['cases'].iloc[i])



In [ ]:
df1['date'] = pd.to_datetime(df.date)

In [ ]:
#Create a new map of GE, so that the video is based on an empty map
map_time = folium.Map([51.2, 9.8], zoom_start=5.8,tiles='Stamen Toner')


#assaults15quart = assaults[(assaults['DateTime'] > "2015-01-01") & (assaults['DateTime'] < "2015-03-31")]
df1 = df1[(df1['date'] > '2020-01-20')]
# Ensure you're handing it floats
df1['Lat'] = df1['Lat'].astype(float)
df1['Lon'] = df1['Lon'].astype(float)

# Filter the DF for columns 

heat_df1 = df1[['Lat', 'Lon']]

# Create weight column, using Hour, 
heat_df1['Weight'] = df1.date.dt.dayofyear.astype(float)
heat_df1['Weight'] = heat_df1['Weight'].astype(float)
heat_df1 = heat_df1.dropna(axis=0, subset=['Lat','Lon', 'Weight']) #And drop Na's 

# List comprehension to make out list of lists. range is 24 due to 24 hours. 
heat_data = [[[row['Lat'],row['Lon']] for index, row in heat_df1[heat_df1['Weight'] == i].iterrows()] for i in range(heat_df1['Weight'].astype(int).min(),heat_df1['Weight'].astype(int).max())]


# Plot it on the map
hm = plugins.HeatMapWithTime(heat_data,auto_play=True,max_opacity=0.8)
hm.add_to(map_time)
# Display the map
map_time

In [ ]:
df2 = df

In [ ]:
df2 = df2.drop_duplicates(subset='county', keep="first")

In [ ]:
#creating a map that's centered to our sample
some_map = folium.Map([51.2, 9.8], zoom_start=5.8,tiles='Stamen Toner')

mc = MarkerCluster()

df2['Total Cases'] = df.groupby('county')['cases'].sum().values
df2['Total Deaths'] = df.groupby('county')['deaths'].sum().values

#creating a Marker for each point in df_sample.  Each point will get a popup with their zip
for row in df2.itertuples():
    mc.add_child(folium.Marker(location=[row.Lat, row.Lon],
                    tooltip=row.county))

#state_loc['Total Cases'] = df.groupby('state')['cases'].sum().values
#state_loc['Total Deaths'] = df.groupby('state')['deaths'].sum().values

#for i in range(state_loc.shape[0]):
#    icon = folium.Icon(color="red")
#    folium.Marker([df2['Long'][i],state_loc['Lat'][i]],
#                  tooltip='State: {} <br>' 'Total Cases: {} <br>' 'Total Deaths: {}'.format(state_loc['County'][i],state_loc['Total Cases'][i],state_loc['Total Deaths'][i]),icon=icon).add_to(map_DE_heat)



    
some_map.add_child(mc)
some_map

In [ ]:
#from future import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import cufflinks as cf
import plotly.offline

In [ ]:
#pip install cufflinks

In [ ]:
#import pandas as pd
#datafile = './data/covid_de.csv'
#Read csv file using pandas
#df = pd.read_csv(datafile)
#df = df.dropna()
#df['date'] = pd.to_datetime(df.date)
#df.date = pd.to_datetime(df["date"], format="%Y-%m-%dT%H:%M:%S.%fZ")
#df = df.sort_values(by='date')

In [ ]:

#cf.go_offline()
#cf.set_config_file(offline=False, world_readable=True)


#def niceplot(State,Value):
    
#    if Value == 'Total':
#        df1 = df[df.state == State]
#        df1 = df1.sort_values(by='date')
#        df1 = df1[['date','cases','deaths','recovered']]
#        df1['Total Cases'] = df1['cases'].cumsum()
#        df1['Total Deaths'] = df1['deaths'].cumsum()
#        df1['Total Recovered'] = df1['recovered'].cumsum()
#        df1 = df1.drop(['cases','deaths','recovered'],axis=1)
#        df1 = df1.set_index('date')
#        df1.iplot(xTitle="Date",
#                    yTitle="Occurrences", title="Corona Development in {}".format(State))
#    if Value == 'By day':
#        df1 = df[df.state == State]
#        df1 = df1.sort_values(by='date')
#        df1 = df1[['date','cases','deaths','recovered']]
#        df1 = df1.set_index('date')
#        df1.columns = ['Cases','Deaths','Recovered']
#        df1.iplot(xTitle="Date",
#                    yTitle="Occurrences", title="Corona Development in {}".format(State), kind='bar')


#interact(niceplot, State=df.state.unique(),Value = ['Total','By day']);

In [ ]:
#import pandas as pd
#import folium
#from folium.plugins import MarkerCluster
#from folium.plugins import FastMarkerCluster

In [ ]:
#file_url = 'http://www2.census.gov/geo/docs/maps-data/data/gazetteer/2016_Gazetteer/2016_Gaz_zcta_national.zip'

#Pandas usually infers zips are numerics, but we lose our leading zeroes so let's go with the object dtype
#df = pd.read_csv(file_url, sep='\t', dtype={'GEOID' : object}) 

#df.columns = df.columns.str.strip() #some column names have some padding
#print (len(df))
#df.head()

In [ ]:
#grab a random sample from df
#subset_of_df = df.sample(n=500)

In [ ]:
#creating a map that's centered to our sample
#some_map = folium.Map(location=[subset_of_df['INTPTLAT'].mean(), 
#                                subset_of_df['INTPTLONG'].mean()], 
#                                zoom_start=4)

#creating a Marker for each point in df_sample.  Each point will get a popup with their zip
#for row in subset_of_df.itertuples():
#    some_map.add_child(folium.Marker(location=[row.INTPTLAT, row.INTPTLONG],
#                                     popup=row.GEOID,row.ALAND))
#some_map

In [ ]:
#creating a map that's centered to our sample
#some_map = folium.Map(location=[subset_of_df['INTPTLAT'].mean(), 
#                                subset_of_df['INTPTLONG'].mean()], 
#                                zoom_start=4)

#mc = MarkerCluster()

#creating a Marker for each point in df_sample.  Each point will get a popup with their zip
#for row in subset_of_df.itertuples():
#    mc.add_child(folium.Marker(location=[row.INTPTLAT, row.INTPTLONG],
#                    popup=row.GEOID))
    
#some_map.add_child(mc)
#some_map

In [ ]:
#creating a map that's centered to our sample
#some_map = folium.Map(location=[subset_of_df['INTPTLAT'].mean(), 
#                                subset_of_df['INTPTLONG'].mean()], 
#                                zoom_start=4)

#some_map.add_child(FastMarkerCluster(subset_of_df[['INTPTLAT', 'INTPTLONG']].values.tolist()))
#some_map

In [ ]:
#creating a map that's centered to our sample
#some_map = folium.Map(location=[subset_of_df['INTPTLAT'].mean(), 
#                                subset_of_df['INTPTLONG'].mean()], 
#                                zoom_start=4)

#callback = ('function (row) {' 
#                'var circle = L.circle(new L.LatLng(row[0], row[1]), {color: "red",  radius: 20000});'
#                'return circle};')

#some_map.add_child(FastMarkerCluster(subset_of_df[['INTPTLAT', 'INTPTLONG']].values.tolist(), callback=callback))
#some_map

In [ ]:
#grab a random sample from df
#subset_of_df = df.sample(n=5000)

#creating a map that's centered to our sample
#some_map = folium.Map(location=[subset_of_df['INTPTLAT'].mean(), 
#                                subset_of_df['INTPTLONG'].mean()], 
#                                zoom_start=4)

#callback = ('function (row) {' 
#                'var circle = L.circle(new L.LatLng(row[0], row[1]), {color: "red",  radius: 10000});'
#                'return circle};')

#some_map.add_child(FastMarkerCluster(subset_of_df[['INTPTLAT', 'INTPTLONG']].values.tolist(), callback=callback))
#some_map